In [24]:
import pandas as pd

data = pd.read_csv('lovesickgirl_1080p_test.mp4loc_test_conf0.75.csv', header=None, names=['frame', 'class', 'x', 'y', 'w', 'h'])
data

,frame,class,x,y,w,h
0,1,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
16019,5873,NaN,NaN,NaN,NaN,NaN
16020,5874,NaN,NaN,NaN,NaN,NaN
16021,5875,NaN,NaN,NaN,NaN,NaN
16022,5876,NaN,NaN,NaN,NaN,NaN


In [25]:
# 각 멤버 csv파일 만들기

rose, lisa, jennie, jisoo = [], [], [], []
temp = []

for idx, rw in data.iterrows():
    temp.append(rw)

for i, x in enumerate(temp):
    if x[1] == 0.0:
        rose.append([x[0], x[1], x[2], x[3], x[4], x[5]])
    elif x[1] == 1.0:
        lisa.append([x[0], x[1], x[2], x[3], x[4], x[5]])
    elif x[1] == 2.0:
        jennie.append([x[0], x[1], x[2], x[3], x[4], x[5]])
    elif x[1] == 3.0:
        jisoo.append([x[0], x[1], x[2], x[3], x[4], x[5]])

        
empty = []

for x in range(5877):    # frame
    empty.append([x+1,"-",0,0,0,0]) # frame, name,x,y,w,h

cf1 = pd.DataFrame(empty, columns=["frame","name","x","y","w","h"])
cf2 = pd.DataFrame(empty, columns=["frame","name","x","y","w","h"])
cf3 = pd.DataFrame(empty, columns=["frame","name","x","y","w","h"])
cf4 = pd.DataFrame(empty, columns=["frame","name","x","y","w","h"])

cf1["name"] = "rose"
cf2["name"] = "lisa"
cf3["name"] = "jennie"
cf4["name"] = "jisoo"

In [26]:
cf1

,frame,name,x,y,w,h
0,1,rose,0,0,0,0
1,2,rose,0,0,0,0
2,3,rose,0,0,0,0
3,4,rose,0,0,0,0
4,5,rose,0,0,0,0
...,...,...,...,...,...,...
5872,5873,rose,0,0,0,0
5873,5874,rose,0,0,0,0
5874,5875,rose,0,0,0,0
5875,5876,rose,0,0,0,0


In [27]:
# detect된 멤버 좌표 행별로 넣는 함수
def frame(df, memlist):
    for row in memlist:
        df.iloc[int(row[0]-1),2] = row[2] # x좌표
        df.iloc[int(row[0]-1),3] = row[3] # y좌표
        df.iloc[int(row[0]-1),4] = row[4] # Width
        df.iloc[int(row[0]-1),5] = row[5] # height

In [28]:
frame(cf1,rose)

### 멤버별 초기값 설정
   - 제일 처음 detect된 좌표를 초기값으로 설정

In [50]:
#로제
for row in range(138):
    cf1.iloc[row,2] = cf1.iloc[138,2]
    cf1.iloc[row,3] = cf1.iloc[138,3]
    cf1.iloc[row,4] = cf1.iloc[138,4]
    cf1.iloc[row,5] = cf1.iloc[138,5]

In [ ]:
# 리사
for row in range(142):
    cf2.iloc[row,2] = cf2.iloc[142,2]
    cf2.iloc[row,3] = cf2.iloc[142,3]
    cf2.iloc[row,4] = cf2.iloc[142,4]
    cf2.iloc[row,5] = cf2.iloc[142,5]

In [ ]:
# 제니
for row in range(143):
    cf3.iloc[row,2] = cf3.iloc[143,2]
    cf3.iloc[row,3] = cf3.iloc[143,3]
    cf3.iloc[row,4] = cf3.iloc[143,4]
    cf3.iloc[row,5] = cf3.iloc[143,5]

In [ ]:
# 지수
for row in range(144):
    cf4.iloc[row,2] = cf4.iloc[144,2]
    cf4.iloc[row,3] = cf4.iloc[144,3]
    cf4.iloc[row,4] = cf4.iloc[144,4]
    cf4.iloc[row,5] = cf4.iloc[144,5]

### 선형증가로 null값 채우고 이동평균값 구해서 튀는값 줄이기

In [ ]:
def moving_avg_frame(df):
    for idx, row in df.iterrows():
        left_idx = 0
        right_idx = 0
        
        x = []
        y = []
        w = []
        h = []
        
        L_x = []
        L_y = []
        L_w = []
        L_h = []
        
        if idx <1: #2frame 부터
            pass
        elif idx == len(df)-2: # 마지막 -2frame 까지
            break
            
        x.append(df.iloc[idx-1,2])
        y.append(df.iloc[idx-1,3])
        w.append(df.iloc[idx-1,4])
        h.append(df.iloc[idx-1,5])
        
        left_idx = idx-1
        
        if row["w"] == 0: # 인식못한 frame이면
            R_idx_list = []
            for i in range(len(df)-(idx+1)):
                
                idx_w = 0
                if df.iloc[idx+i+1,4] != 0:
                    idx_w = idx+i+1
                    L_x.append(df.iloc[idx_w,2])
                    L_y.append(df.iloc[idx_w,3])
                    L_w.append(df.iloc[idx_w,4])
                    L_h.append(df.iloc[idx_w,5])
                    R_idx_list.append(idx_w)
                    
                    for j in range(1,6):
                        if idx_w + j == len(df)-1:
                            break
                            
                        elif df.iloc[idx_w+j,4] !=0:
                            if len(L_x) == 3 or (len(L_x)<2 and j ==3):
                                break
                                
                            L_x.append(df.iloc[idx_w+j,2])
                            L_y.append(df.iloc[idx_w+j,3])
                            L_w.append(df.iloc[idx_w+j,4])
                            L_h.append(df.iloc[idx_w+j,5])
                            R_idx_list.append(idx_w+j)
                            
                        else:
                            pass
                        
                    break
            print(L_x,L_y,L_w,L_h)
            
            if len(L_x) == 1:
                x.append(L_x[0])
                y.append(L_y[0])
                w.append(L_w[0])
                h.append(L_h[0])
            elif len(L_x) == 2:
                if abs(x[0]-L_x[0]) <abs(x[0]-L_x[1]):
                    x.append(L_x[0])
                    y.append(L_y[0])
                    w.append(L_w[0])
                    h.append(L_h[0])
                    right_idx = R_idx_list[0]
                    
                else:
                    x.append(L_x[1])
                    y.append(L_y[1])
                    w.append(L_w[1])
                    h.append(L_h[1])
                    right_idx = R_idx_list[1]
                    
            elif len(L_x) == 3:
                if abs(L_x[0]-L_x[1]) < abs(L_x[1]-L_x[2]) and abs(L_x[0]-L_x[1]) < abs(L_x[2]-L_x[0]):
                    x.append(L_x[0])
                    y.append(L_y[0])
                    w.append(L_w[0])
                    h.append(L_h[0])  
                    right_idx = R_idx_list[0]
                
                elif abs(L_x[1]-L_x[2]) < abs(L_x[0]-L_x[1]) and abs(L_x[1]-L_x[2]) < abs(L_x[2]-L_x[0]):
                    x.append(L_x[1])
                    y.append(L_y[1])
                    w.append(L_w[1])
                    h.append(L_h[1])  
                    right_idx = R_idx_list[1]
                    
                else:
                    x.append(L_x[2])
                    y.append(L_y[2])
                    w.append(L_w[2])
                    h.append(L_h[2]) 
                    right_idx = R_idx_list[2]
                    
            zero_len = right_idx - left_idx
            
            if len(x) == 1:
                return
            
            div_val_x = (x[1] - x[0])/zero_len # 차이를 왼쪽 오른쪽 인덱스 길이만큼 나눈값
            div_val_y = (y[1] - y[0])/zero_len
            div_val_w = (w[1] - w[0])/zero_len
            div_val_h = (h[1] - h[0])/zero_len
            
            #선형 증가값 적용
            for k in range(zero_len):
                df.iloc[idx+k, 2] = x[0] + div_val_x*(k+1)
                df.iloc[idx+k, 3] = y[0] + div_val_y*(k+1)
                df.iloc[idx+k, 4] = w[0] + div_val_w*(k+1)
                df.iloc[idx+k, 5] = h[0] + div_val_h*(k+1)
                
            for s in range(zero_len): # 선형증가한 위치만큼 뒤로 이동
                pass
                            
                            
            

In [52]:
moving_avg_frame(cf1)
moving_avg_frame(cf2)
moving_avg_frame(cf3)
moving_avg_frame(cf4)

[0.32369792461395264, 0.3200520873069763, 0.31692707538604736] [0.3819444477558136, 0.3819444477558136, 0.3787037134170532] [0.02656250074505806, 0.02760416641831398, 0.02968749962747097] [0.0694444477558136, 0.07314814627170563, 0.07592592388391495]
[0.35104167759418486, 0.3466145932674408, 0.3421875089406967] [0.38453704714775083, 0.3841666758060455, 0.3837963044643402] [0.028333333693444727, 0.02822916693985462, 0.028125000186264515] [0.06796296313405037, 0.06870370358228683, 0.0694444440305233]
[0.3466145932674408, 0.3421875089406967, 0.3377604246139526] [0.3841666758060455, 0.3837963044643402, 0.3834259331226349] [0.02822916693985462, 0.028125000186264515, 0.02802083343267441] [0.06870370358228683, 0.0694444440305233, 0.07018518447875977]
[0.3421875089406967, 0.3377604246139526, 0.33333334028720857] [0.3837963044643402, 0.3834259331226349, 0.3830555617809296] [0.028125000186264515, 0.02802083343267441, 0.027916666679084302] [0.0694444440305233, 0.07018518447875977, 0.0709259249269

[0.546093761920929, 0.5463541746139526, 0.5453125238418579] [0.3148148059844971, 0.3157407343387604, 0.3171296417713165] [0.02656250074505806, 0.02708333358168602, 0.02812499925494194] [0.05370370298624039, 0.05370370298624039, 0.054629631340503686]
[0.5471874952316285, 0.5470312476158142, 0.546875] [0.3141666561365128, 0.31425924897193913, 0.3143518418073654] [0.02802083399146795, 0.027812500670552253, 0.027604167349636555] [0.05888888873159885, 0.05814814791083336, 0.05740740709006786]
[0.5470312476158142, 0.546875, 0.5467187523841858] [0.3142592489719391, 0.3143518418073654, 0.31444443464279176] [0.027812500670552253, 0.027604167349636555, 0.027395834028720856] [0.05814814791083336, 0.05740740709006786, 0.05666666626930237]
[0.546875, 0.5467187523841858, 0.5465625047683715] [0.3143518418073654, 0.31444443464279176, 0.3145370274782181] [0.027604167349636555, 0.027395834028720856, 0.027187500707805158] [0.05740740709006786, 0.05666666626930237, 0.055925925448536876]
[0.546718752384185

[0.6566406488418579, 0.6638021071751913, 0.6709635655085245] [0.3446759283542633, 0.3452160557111104, 0.3457561830679576] [0.02526041679084301, 0.02482638880610466, 0.02439236082136631] [0.06620370224118233, 0.06388888756434123, 0.06157407288750012]
[0.6638021071751913, 0.6709635655085245, 0.6781250238418579] [0.3452160557111104, 0.3457561830679576, 0.3462963104248047] [0.02482638880610466, 0.02439236082136631, 0.02395833283662796] [0.06388888756434123, 0.06157407288750012, 0.05925925821065903]
[0.6709635655085245, 0.6781250238418579, 0.680468738079071] [0.3457561830679576, 0.3462963104248047, 0.3435185253620148] [0.02439236082136631, 0.02395833283662796, 0.02864583395421505] [0.06157407288750012, 0.05925925821065903, 0.06851851940155029]
[0.6781250238418579, 0.680468738079071, 0.6869791746139526] [0.3462963104248047, 0.3435185253620148, 0.34166666865348816] [0.02395833283662796, 0.02864583395421505, 0.03125] [0.05925925821065903, 0.06851851940155029, 0.07592592388391495]
[0.5958333611

[0.5153645873069763, 0.5166666507720947, 0.516406238079071] [0.4134259223937988, 0.38749998807907104, 0.3615740835666657] [0.03281249850988388, 0.03125, 0.03489583358168602] [0.07685185223817825, 0.07870370149612428, 0.08055555820465088]
[0.5249881744384766, 0.524550199508667, 0.5241122245788574] [0.5242424363439734, 0.5170454653826627, 0.509848494421352] [0.03439867445691065, 0.0342329547486522, 0.034067235040393745] [0.07710437713698907, 0.07718855210325935, 0.07727272706952962]
[0.524550199508667, 0.5241122245788574, 0.5236742496490479] [0.5170454653826627, 0.509848494421352, 0.5026515234600414] [0.0342329547486522, 0.034067235040393745, 0.033901515332135285] [0.07718855210325934, 0.07727272706952962, 0.0773569020357999]
[0.5241122245788574, 0.5236742496490479, 0.5232362747192383] [0.509848494421352, 0.5026515234600414, 0.4954545524987307] [0.034067235040393745, 0.033901515332135285, 0.03373579562387683] [0.07727272706952962, 0.0773569020357999, 0.07744107700207017]
[0.5236742496490

[0.6382812261581421, 0.6348958611488342, 0.6213541626930237] [0.2726851999759674, 0.26944443583488464, 0.2532407343387604] [0.05885416641831397, 0.05624999850988388, 0.0572916679084301] [0.08796296268701552, 0.08888889104127884, 0.09351851791143416]
[0.629867801299462, 0.6307091437853299, 0.631550486271198] [0.293340456027251, 0.29127493042212266, 0.2892094048169943] [0.04923878237605095, 0.05020032078027725, 0.051161859184503555] [0.0794159546494484, 0.08027065545320511, 0.08112535625696182]
[0.6307091437853299, 0.631550486271198, 0.632391828757066] [0.29127493042212266, 0.2892094048169943, 0.28714387921186596] [0.05020032078027725, 0.051161859184503555, 0.05212339758872986] [0.08027065545320511, 0.08112535625696182, 0.08198005706071854]
[0.631550486271198, 0.632391828757066, 0.633233171242934] [0.2892094048169943, 0.28714387921186596, 0.28507835360673756] [0.051161859184503555, 0.05212339758872986, 0.05308493599295616] [0.08112535625696182, 0.08198005706071854, 0.08283475786447524]
[

...

[0.8229538713182721, 0.8224082362084162, 0.8218626010985601] [0.43802909765924725, 0.4367504375321524, 0.43547177740505766] [0.026711309860859598, 0.026959325674743878, 0.02720734148862816] [0.06388889253139496, 0.06388889253139496, 0.06388889253139496]
[0.8224082362084162, 0.8218626010985601, 0.8213169659887042] [0.4367504375321524, 0.43547177740505766, 0.43419311727796284] [0.026959325674743878, 0.02720734148862816, 0.027455357302512442] [0.06388889253139496, 0.06388889253139496, 0.06388889253139496]
[0.8218626010985601, 0.8213169659887042, 0.8207713308788481] [0.43547177740505766, 0.43419311727796284, 0.432914457150868] [0.02720734148862816, 0.027455357302512442, 0.027703373116396722] [0.06388889253139496, 0.06388889253139496, 0.06388889253139496]
[0.8213169659887042, 0.8207713308788481, 0.8202256957689921] [0.43419311727796284, 0.432914457150868, 0.4316357970237732] [0.027455357302512442, 0.027703373116396722, 0.027951388930281002] [0.06388889253139496, 0.06388889253139496, 0.06388

[0.5184548815091451, 0.5157118240992228, 0.5129687666893006] [0.421666665871938, 0.4217592577139536, 0.4218518495559692] [0.029201389476656913, 0.02934027835726738, 0.029479167237877843] [0.060864197959502535, 0.060802469650904335, 0.060740741342306136]
[0.5157118240992228, 0.5129687666893006, 0.5102257092793783] [0.4217592577139536, 0.4218518495559692, 0.4219444413979848] [0.02934027835726738, 0.029479167237877843, 0.02961805611848831] [0.060802469650904335, 0.060740741342306136, 0.060679013033707936]
[0.5129687666893006, 0.5102257092793783, 0.507482651869456] [0.4218518495559692, 0.4219444413979848, 0.42203703324000036] [0.029479167237877843, 0.02961805611848831, 0.029756944999098776] [0.060740741342306136, 0.060679013033707936, 0.06061728472510973]
[0.5102257092793783, 0.507482651869456, 0.5047395944595336] [0.42194444139798476, 0.42203703324000036, 0.42212962508201596] [0.029618056118488313, 0.029756944999098776, 0.029895833879709243] [0.06067901303370793, 0.06061728472510973, 0.06

[0.4253645807504654, 0.4285677060484886, 0.43177083134651184] [0.3659722238779068, 0.3649305567145348, 0.3638888895511627] [0.02937500011175871, 0.02927083345130086, 0.029166666790843013] [0.0686111144721508, 0.06763889230787754, 0.06666667014360428]
[0.4285677060484886, 0.43177083134651184, 0.4346354305744171] [0.3649305567145348, 0.3638888895511627, 0.3601851761341095] [0.02927083345130086, 0.029166666790843013, 0.03177083283662796] [0.06763889230787754, 0.06666667014360428, 0.06481481343507768]
[0.43177083134651184, 0.4346354305744171, 0.4403645694255829] [0.3638888895511627, 0.3601851761341095, 0.3592592477798462] [0.029166666790843013, 0.03177083283662796, 0.030729167163372036] [0.06666667014360428, 0.06481481343507768, 0.06296296417713165]
[0.4994791746139526, 0.5, 0.5] [0.395370364189148, 0.40138888359069824, 0.4078703820705414] [0.029166666790843013, 0.03333333507180214, 0.03333333507180214] [0.07592592388391495, 0.08055555820465088, 0.0824074074625969]
[0.4650579580376225, 0.4

[0.4887474780121157, 0.4893397160114781, 0.4899319540108404] [0.39684885692211885, 0.39708780569414937, 0.3973267544661799] [0.03317372386734332, 0.033182124457051675, 0.03319052504676004] [0.07942055153750605, 0.07948028873051366, 0.07954002592352129]
[0.4893397160114781, 0.4899319540108404, 0.4905241920102027] [0.39708780569414937, 0.3973267544661799, 0.39756570323821033] [0.033182124457051675, 0.03319052504676004, 0.033198925636468396] [0.07948028873051366, 0.07954002592352129, 0.07959976311652892]
[0.4899319540108404, 0.4905241920102027, 0.49111643000956506] [0.3973267544661798, 0.39756570323821033, 0.39780465201024084] [0.03319052504676004, 0.033198925636468396, 0.03320732622617675] [0.07954002592352129, 0.07959976311652892, 0.07965950030953653]
[0.4905241920102027, 0.49111643000956506, 0.4917086680089274] [0.39756570323821033, 0.39780465201024084, 0.39804360078227136] [0.033198925636468396, 0.03320732622617675, 0.03321572681588511] [0.07959976311652892, 0.07965950030953653, 0.079

[0.457812488079071, 0.46484375] [0.3939814865589142, 0.38009259104728704] [0.03437500074505806, 0.03802083432674408] [0.07314814627170563, 0.08055555820465088]
[0.46484375, 0.4705729186534882, 0.4710937440395355] [0.38009259104728704, 0.3814814686775208, 0.38333332538604736] [0.03802083432674408, 0.03489583358168602, 0.0338541679084301] [0.08055555820465088, 0.08148147910833359, 0.07777778059244156]
[0.4641927033662796, 0.4663194417953491, 0.4684461802244187] [0.38773147761821747, 0.3856481413046519, 0.38356480499108636] [0.03463541716337204, 0.034722222636143364, 0.034809028108914696] [0.07731481269001961, 0.07870370149612427, 0.08009259030222893]
[0.4663194417953491, 0.4684461802244187, 0.4705729186534882] [0.3856481413046519, 0.38356480499108636, 0.3814814686775208] [0.034722222636143364, 0.034809028108914696, 0.03489583358168602] [0.07870370149612427, 0.08009259030222893, 0.08148147910833359]
[0.4684461802244187, 0.4705729186534882, 0.4710937440395355] [0.38356480499108636, 0.38148

### 마지막 값 채우기
   - 마지막으로 detect된 좌표로 마지막 채우기

In [62]:
def tail_fill_frame(df):
    for idx,row in df.iterrows():
        if row["x"] == 0:
            df.iloc[idx,2] = df.iloc[idx-1,2]
            df.iloc[idx,3] = df.iloc[idx-1,3]
            df.iloc[idx,4] = df.iloc[idx-1,4]
            df.iloc[idx,5] = df.iloc[idx-1,5]

In [64]:
tail_fill_frame(cf1)
tail_fill_frame(cf2)
tail_fill_frame(cf3)
tail_fill_frame(cf4)

[0.61328125, 0.6166666746139526] [0.3995370268821716, 0.4018518626689911] [0.03177083283662796, 0.03125] [0.07870370149612428, 0.07592592388391495]
[0.6109375059604645, 0.61328125, 0.6166666746139526] [0.3978009149432182, 0.3995370268821716, 0.4018518626689911] [0.02968749962747097, 0.03177083283662796, 0.03125] [0.07662036828696729, 0.07870370149612428, 0.07592592388391495]
[0.61328125, 0.6166666746139526] [0.3995370268821716, 0.4018518626689911] [0.03177083283662796, 0.03125] [0.07870370149612428, 0.07592592388391495]
[0.6565104126930237, 0.6591145992279053, 0.6651041507720947] [0.3893518447875977, 0.39027777314186096, 0.39027777314186096] [0.02864583395421505, 0.02656250074505806, 0.02708333358168602] [0.07500000298023224, 0.07314814627170563, 0.07314814627170563]
[0.6299479206403097, 0.6343750026490953, 0.638802084657881] [0.39768519004186, 0.3962962991661496, 0.3949074082904392] [0.030381944651405018, 0.030092592868540022, 0.029803241085675027] [0.07561728358268738, 0.075514403482

[0.672656238079071, 0.6734374910593033, 0.6742187440395355] [0.3421296179294586, 0.3395833224058151, 0.33703702688217163] [0.03828124888241291, 0.03782551968470216, 0.037369790486991405] [0.05370370391756296, 0.05393518530763686, 0.05416666669771075]
[0.6734374910593033, 0.6742187440395355, 0.6749999970197678] [0.3395833224058151, 0.33703702688217163, 0.33449073135852814] [0.03782551968470216, 0.037369790486991405, 0.03691406128928065] [0.05393518530763686, 0.05416666669771075, 0.05439814808778465]
[0.6742187440395355, 0.6749999970197678, 0.67578125] [0.33703702688217163, 0.33449073135852814, 0.33194443583488464] [0.037369790486991405, 0.03691406128928065, 0.0364583320915699] [0.05416666669771075, 0.05439814808778465, 0.05462962947785854]
[0.6749999970197678, 0.67578125, 0.6765625029802322] [0.33449073135852814, 0.33194443583488464, 0.32939814031124115] [0.03691406128928065, 0.0364583320915699, 0.03600260289385915] [0.05439814808778465, 0.05462962947785854, 0.054861110867932446]
[0.675

[0.5924479365348816, 0.5859375, 0.5791666507720947] [0.34398147463798523, 0.3393518626689911, 0.33472222089767456] [0.03489583358168602, 0.03541666641831398, 0.03333333507180214] [0.07314814627170563, 0.06759259104728699, 0.07129629701375961]
[0.4731770753860474, 0.4450520873069763, 0.4385416805744171] [0.3393518626689911, 0.3384259343147278, 0.33888888359069824] [0.030729167163372036, 0.03177083283662796, 0.03125] [0.0694444477558136, 0.0694444477558136, 0.07222222536802292]
[0.5348437666893006, 0.5273611267407735, 0.5198784867922465] [0.3287963032722473, 0.32959877252578734, 0.3304012417793274] [0.03218749910593033, 0.0321527769168218, 0.032118054727713265] [0.07092592716217042, 0.070802470544974, 0.0706790139277776]
[0.5273611267407735, 0.5198784867922465, 0.5123958468437195] [0.32959877252578734, 0.3304012417793274, 0.33120371103286744] [0.03215277691682179, 0.032118054727713265, 0.032083332538604736] [0.070802470544974, 0.0706790139277776, 0.0705555573105812]
[0.5198784867922465, 

[0.20130208283662795, 0.20295138855775197, 0.20460069427887598] [0.48138890266418455, 0.47333334684371947, 0.4652777910232544] [0.03322916813194752, 0.03319444581866264, 0.03315972350537777] [0.045555555820465096, 0.046543210248152424, 0.04753086467583975]
[0.20295138855775197, 0.20460069427887598, 0.20625] [0.47333334684371947, 0.4652777910232544, 0.4572222352027893] [0.03319444581866264, 0.03315972350537777, 0.0331250011920929] [0.046543210248152424, 0.04753086467583975, 0.04851851910352707]
[0.20460069427887598, 0.20625, 0.207899305721124] [0.4652777910232544, 0.4572222352027893, 0.44916667938232424] [0.03315972350537777, 0.0331250011920929, 0.03309027887880802] [0.047530864675839744, 0.04851851910352707, 0.0495061735312144]
[0.20625, 0.207899305721124, 0.209548611442248] [0.4572222352027893, 0.44916667938232424, 0.4411111235618591] [0.0331250011920929, 0.03309027887880802, 0.03305555656552315] [0.04851851910352707, 0.0495061735312144, 0.05049382795890173]
[0.207899305721124, 0.2095

[0.5289062261581421, 0.5299479365348816, 0.53125] [0.3199073970317841, 0.3157407343387604, 0.3222222328186035] [0.0338541679084301, 0.03281249850988388, 0.029166666790843013] [0.07500000298023224, 0.08703703433275223, 0.08518518507480621]
[0.53515625, 0.5372396111488342, 0.5364583134651184] [0.3254629671573639, 0.3166666626930237, 0.31388887763023376] [0.02760416641831398, 0.02968749962747097, 0.03020833246409893] [0.06203703582286835, 0.07407407462596892, 0.07407407462596892]
[0.5361979263169425, 0.5364583475249154, 0.5367187687328884] [0.3238095300538199, 0.3220238132136209, 0.3202380963734218] [0.032366071694663594, 0.031696428677865436, 0.031026785661067282] [0.07354497270924704, 0.07367724818842751, 0.07380952366760798]
[0.5364583475249154, 0.5367187687328884, 0.5369791899408612] [0.3220238132136209, 0.3202380963734218, 0.31845237953322275] [0.031696428677865436, 0.031026785661067282, 0.030357142644269128] [0.07367724818842751, 0.07380952366760798, 0.07394179914678845]
[0.53671876

...

[0.5104910714285714, 0.5033482142857143, 0.49620535714285713] [0.5622354388237, 0.5575396716594696, 0.5528439044952392] [0.02744047631110464, 0.027529761993459294, 0.027619047675813947] [0.04203703701496124, 0.04259259253740311, 0.043148148059844973]
[0.5033482142857143, 0.49620535714285713, 0.4890625] [0.5575396716594696, 0.5528439044952392, 0.5481481373310089] [0.027529761993459294, 0.027619047675813947, 0.0277083333581686] [0.04259259253740311, 0.043148148059844973, 0.04370370358228683]
[0.49620535714285713, 0.4890625, 0.48191964285714284] [0.5528439044952392, 0.5481481373310089, 0.5434523701667786] [0.027619047675813947, 0.0277083333581686, 0.027797619040523257] [0.043148148059844973, 0.04370370358228683, 0.0442592591047287]
[0.4890625, 0.48191964285714284, 0.4747767857142857] [0.5481481373310089, 0.5434523701667786, 0.5387566030025482] [0.0277083333581686, 0.027797619040523257, 0.02788690472287791] [0.04370370358228683, 0.0442592591047287, 0.04481481462717056]
[0.48191964285714284

[0.3598415752251943, 0.3629774252573649, 0.3661132752895355] [0.40715664873520535, 0.4074845810731252, 0.4078125134110451] [0.0263671869567285, 0.026302082774539787, 0.02623697859235108] [0.05520833500971396, 0.0550925942758719, 0.05497685354202985]
[0.3629774252573649, 0.3661132752895355, 0.3692491253217062] [0.4074845810731252, 0.4078125134110451, 0.40814044574896496] [0.026302082774539787, 0.02623697859235108, 0.02617187441016237] [0.0550925942758719, 0.05497685354202985, 0.0548611128081878]
[0.3661132752895355, 0.3692491253217062, 0.37238497535387677] [0.4078125134110451, 0.40814044574896496, 0.4084683780868848] [0.02623697859235108, 0.02617187441016237, 0.02610677022797366] [0.05497685354202985, 0.0548611128081878, 0.05474537207434574]
[0.3692491253217062, 0.37238497535387677, 0.37552082538604736] [0.40814044574896496, 0.4084683780868848, 0.4087963104248047] [0.02617187441016237, 0.02610677022797366, 0.02604166604578495] [0.0548611128081878, 0.05474537207434574, 0.0546296313405036

[0.5293738286603581, 0.5299479365348816, 0.5231770873069763] [0.36499369754032657, 0.36342594027519226, 0.3847222328186035] [0.027687026314776052, 0.02760416641831398, 0.03281249850988388] [0.060879630967974656, 0.060185186564922326, 0.06759259104728699]
[0.5231770873069763, 0.5200520753860474, 0.5192708373069763] [0.3847222328186035, 0.4009259343147278, 0.4185185134410858] [0.03281249850988388, 0.03177083283662796, 0.03125] [0.06759259104728699, 0.07037036865949631, 0.07222222536802292]
[0.5225260406732559, 0.5200520753860474, 0.5192708373069763] [0.3915509358048439, 0.4009259343147278, 0.4185185134410858] [0.030729166232049465, 0.03177083283662796, 0.03125] [0.06782407313585281, 0.07037036865949631, 0.07222222536802292]
[0.641406238079071, 0.6466146111488342, 0.651562511920929] [0.3884259164333344, 0.39120370149612427, 0.3986110985279083] [0.03177083283662796, 0.02968749962747097, 0.03020833246409893] [0.07129629701375961, 0.07314814627170563, 0.07314814627170563]
[0.6617187261581421

[0.5256127680049223, 0.5246783319641563, 0.5237438959233901] [0.39234749359243054, 0.392061545568354, 0.39177559754427743] [0.02971813641488552, 0.029840685427188873, 0.029963234439492226] [0.07216775417327881, 0.07241285219788551, 0.07265795022249222]
[0.5246783319641563, 0.5237438959233901, 0.5228094598826241] [0.392061545568354, 0.39177559754427743, 0.3914896495202008] [0.029840685427188873, 0.029963234439492226, 0.030085783451795578] [0.07241285219788551, 0.07265795022249222, 0.07290304824709892]
[0.5237438959233901, 0.5228094598826241, 0.5218750238418579] [0.39177559754427743, 0.3914896495202008, 0.39120370149612427] [0.029963234439492226, 0.030085783451795578, 0.03020833246409893] [0.07265795022249222, 0.07290304824709892, 0.07314814627170563]
[0.5228094598826241, 0.5218750238418579, 0.5286458134651184] [0.3914896495202008, 0.39120370149612427, 0.39027777314186096] [0.030085783451795578, 0.03020833246409893, 0.02812499925494194] [0.07290304824709892, 0.07314814627170563, 0.069444

[0.8517485090664455, 0.8518601145063128, 0.85197171994618] [0.1986111125775746, 0.19768518848078592, 0.19675926438399724] [0.042038691363164356, 0.04226190543600491, 0.04248511950884547] [0.08505290959562574, 0.08478835863726479, 0.08452380767890386]
[0.8518601145063128, 0.85197171994618, 0.8520833253860474] [0.19768518848078592, 0.19675926438399724, 0.19583334028720856] [0.04226190543600491, 0.04248511950884547, 0.04270833358168602] [0.08478835863726479, 0.08452380767890386, 0.08425925672054291]
[0.85197171994618, 0.8520833253860474, 0.8507812619209291] [0.19675926438399724, 0.19583334028720856, 0.1953703761100769] [0.04248511950884547, 0.04270833358168602, 0.04218750074505806] [0.08452380767890386, 0.08425925672054291, 0.08518518507480621]
[0.8510416746139526, 0.8510416746139526, 0.8513020873069763] [0.23472222685813904, 0.23472222685813904, 0.2337962985038757] [0.046875, 0.046875, 0.04635416716337204] [0.09166666865348816, 0.09166666865348816, 0.09166666865348816]
[0.851041674613952

In [12]:
cf1.to_csv("rose.csv",index=False)
cf2.to_csv("lisa.csv",index=False)
cf3.to_csv("jennie.csv",index=False)
cf4.to_csv("jisoo.csv",index=False)